In [213]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier #,Ridge,Lasso,ElasticNet
from sklearn.metrics import accuracy_score,recall_score,precision_score,roc_auc_score,f1_score
from sklearn.model_selection import train_test_split,cross_val_score,KFold,RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import NearMiss
from sklearn.datasets import make_classification
import numpy as np
from collections import Counter

Opening our DataFrame

In [215]:
df = pd.read_csv('../../data/raw/application_data.csv')

In [217]:
df.shape

(307511, 122)

Let's check the overall info of our DataFrame

In [218]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


Let's drop KidneyAilments and HeartAilments that are full null column values, and since we do not have any other table to join then we can drop Id

In [219]:
df.drop(columns = ['SK_ID_CURR','AMT_GOODS_PRICE'],inplace = True)

We can see below that it has been dropped

In [220]:
df.sample(3)

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
147243,0,Cash loans,F,N,N,0,261000.0,675000.0,26154.0,Family,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
84365,0,Cash loans,F,N,Y,0,166500.0,544491.0,15732.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0
163497,0,Cash loans,F,N,Y,0,112500.0,521280.0,25209.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


We can check the overall statistics from our dataframe

In [221]:
object_cols = df.select_dtypes(include=['object']).columns

<b> Pipeline Team </b> You should create a function that converts all strings to uppercase

In [222]:
df[object_cols] = df[object_cols].apply(lambda x: x.str.upper())

In [223]:
df.describe(include = 'all')

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511,307511,307511,307511,307511.000000,3.075110e+05,3.075110e+05,307499.000000,306219,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
unique,NaN,2,3,2,2,NaN,NaN,NaN,NaN,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,CASH LOANS,F,N,Y,NaN,NaN,NaN,NaN,UNACCOMPANIED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,278232,202448,202924,213312,NaN,NaN,NaN,NaN,248526,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,0.080729,NaN,NaN,NaN,NaN,0.417052,1.687979e+05,5.990260e+05,27108.573909,NaN,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,0.272419,NaN,NaN,NaN,NaN,0.722121,2.371231e+05,4.024908e+05,14493.737315,NaN,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,0.000000,NaN,NaN,NaN,NaN,0.000000,2.565000e+04,4.500000e+04,1615.500000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,NaN,NaN,NaN,NaN,0.000000,1.125000e+05,2.700000e+05,16524.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,NaN,NaN,NaN,NaN,0.000000,1.471500e+05,5.135310e+05,24903.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,NaN,NaN,NaN,NaN,1.000000,2.025000e+05,8.086500e+05,34596.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000


In [224]:
if (df['AMT_ANNUITY'].dtypes == 'int64') | (df['AMT_ANNUITY'].dtypes == 'float64'):
    print('x')

x


<b> Pipeline Team </b> Example of pipeline function

In [225]:
def drop_high_nulls_col(df):
    # Grouped median calculation for filling missing values
    
    for cols in df.columns:
        if (df[cols].notnull().sum() / df.shape[0]) < 0.4:
            df.drop(columns=[cols], inplace=True)
            
    return df

In [226]:
df = drop_high_nulls_col(df)

Let's understand FacilityId better, because if there is lot's of facility id's then we can also drop this column

In [227]:
df.shape[0]

307511

### Feature Engineering

In [228]:
X_col = df.drop(columns = 'TARGET').columns
y_col = ['TARGET']

We are doing a 80-20 Train,Test split we will be locking the random_state as 37 so our results are easy to produce again

In [229]:
group_cols = ['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']

In [306]:
object_cols = df.select_dtypes(include=['object']).columns
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
for name in df[object_cols].columns:
    
    df[name]= label_encoder.fit_transform(df[name])
    print(df[name])

0         0
1         0
2         1
3         0
4         0
         ..
307506    0
307507    0
307508    0
307509    0
307510    0
Name: NAME_CONTRACT_TYPE, Length: 307511, dtype: int32
0         1
1         0
2         1
3         0
4         1
         ..
307506    1
307507    0
307508    0
307509    0
307510    0
Name: CODE_GENDER, Length: 307511, dtype: int32
0         0
1         0
2         1
3         0
4         0
         ..
307506    0
307507    0
307508    0
307509    0
307510    0
Name: FLAG_OWN_CAR, Length: 307511, dtype: int32
0         1
1         0
2         1
3         1
4         1
         ..
307506    0
307507    1
307508    1
307509    1
307510    0
Name: FLAG_OWN_REALTY, Length: 307511, dtype: int32
0         6
1         1
2         6
3         6
4         6
         ..
307506    6
307507    6
307508    6
307509    6
307510    6
Name: NAME_TYPE_SUITE, Length: 307511, dtype: int32
0         7
1         4
2         7
3         7
4         7
         ..
307506    7


In [308]:
X_train,X_test,y_train,y_test = train_test_split(df[X_col],df[y_col],test_size=0.3, random_state=0)

In [309]:
#X_train.to_parquet('../data/processed/X_train.parquet',index=False)
#X_test.to_parquet('../data/processed/X_test.parquet',index=False)
#y_train.to_parquet('../data/processed/y_train.parquet',index=False)
#y_test.to_parquet('../data/processed/y_test.parquet',index=False)

Now to start feature engineering let's check our null values one more time

<b> Pipeline Team </b> Example of pipeline function

In [310]:
def null_dict(df,grouped_cols):
    imputation_dict = {}
    # Calculate the sum of null values for each column
    null_sum = df.isnull().sum()

    # Filter columns where the sum of null values is greater than 0
    columns_with_nulls = null_sum[null_sum > 0].index

    # Select only the columns from the original DataFrame that have more than 0 null values
    null_cols = df.loc[:, columns_with_nulls].columns
    for col in null_cols:
        if (df[col].dtypes == 'int64') | (df[col].dtypes == 'float64'):
            imputation_dict.update({col : df.groupby(grouped_cols)[col].median().to_dict()})
        elif (df[col].dtypes == 'O'):
            imputation_dict.update({col : df.groupby(grouped_cols)[col].agg(pd.Series.mode).to_dict()})
                
    return imputation_dict

Lets go ahead and imput the values on the DF

In [311]:
imput_dict = null_dict(X_train,group_cols)

<b> Pipeline Team </b> Example of pipeline function

In [312]:
def fillnulls(data,grouped_cols,imput_dict):
    imputation_dict = imput_dict
    
    col_x,col_y,col_z =grouped_cols
    
    for col in imputation_dict:
        for val in imputation_dict[col]:
            val_x,val_y,val_z = val
            data.loc[(data[col_x] == val_x) & (data[col_z] == val_z) & (data[col_z] == val_z),col] = imputation_dict[col][val]
    return data

In [313]:
X_train = fillnulls(X_train,group_cols,imput_dict)
X_test = fillnulls(X_test,group_cols,imput_dict)

<b> Pipeline Team </b> You should create a function to do one hotting and one to convert it back

In [314]:
object_cols = df.select_dtypes(include=['object']).columns

In [315]:
oneh = OneHotEncoder(sparse_output=False)
oneh.fit(X_train[object_cols])

OneHotEncoder(sparse_output=False)

In [316]:
col_names = oneh.get_feature_names_out()

In [317]:
#X_train_dummies = pd.DataFrame(oneh.transform(X_train[object_cols]),columns = col_names)
#X_train_dummies.index = X_train.index
#X_train = pd.merge(X_train,X_train_dummies,left_index=True,right_index=True)
#
#X_test_dummies = pd.DataFrame(oneh.transform(X_test[object_cols]),columns = col_names)
#X_test_dummies.index = X_test.index
#X_test = pd.merge(X_test,X_test_dummies,left_index=True,right_index=True)

In [318]:
X_train.sample(3)

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
264970,0,1,1,0,0,157500.0,900000.0,28107.0,6,7,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
195079,0,0,1,1,2,157500.0,805536.0,29398.5,6,7,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
128235,0,1,0,1,0,135000.0,823621.5,29398.5,1,3,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


All that's left is to drop Gender

In [319]:
#X_train.drop(columns = object_cols,inplace = True)
#X_test.drop(columns = object_cols,inplace = True)

We will now transform balance this data

<b> Pipeline Team </b> Create a function that combines the three techniques and outpus X_train, y_train midsampled, check below for what you should expect

In [320]:
y_train.groupby(['TARGET']).size()

TARGET
0    197772
1     17485
dtype: int64

In [321]:
def oversample_with_adasyn(X, y, target_minority_samples=180000):
    """
    Oversample the minority class using ADASYN until the minority class reaches the target_minority_samples.
    
    Parameters:
    - X: Feature matrix
    - y: Target vector
    - target_minority_samples: Desired number of samples for the minority class after oversampling
    
    Returns:
    - X_resampled, y_resampled: Feature matrix and target vector after resampling
    """
    # Identify minority class
    classes, counts = np.unique(y, return_counts=True)
    minority_class = classes[np.argmin(counts)]
    current_minority_samples = counts[np.argmin(counts)]

    # Calculate how many new samples we need to generate for the minority class
    if current_minority_samples >= target_minority_samples:
        raise ValueError("The minority class already has more or equal samples than the target_minority_samples.")
    
    n_samples_to_generate = target_minority_samples - current_minority_samples

    # Set the sampling strategy for ADASYN
    # This will tell ADASYN to generate the specified number of samples for the minority class
    sampling_strategy = {minority_class: n_samples_to_generate}

    # Create the ADASYN instance and fit it to the data
    adasyn = ADASYN(sampling_strategy=sampling_strategy, n_neighbors=5, random_state=42)
    X_resampled, y_resampled = adasyn.fit_resample(X, y)

    return X_resampled, y_resampled

In [322]:
def undersample_with_nearmiss(X, y):
    """
    Undersample the majority class using NearMiss until it is balanced with the minority class.
    
    Parameters:
    - X: Feature matrix
    - y: Target vector
    
    Returns:
    - X_resampled, y_resampled: Feature matrix and target vector after resampling
    """
    # Identify majority and minority classes
    classes, counts = np.unique(y, return_counts=True)
    minority_class_count = min(counts)
    
    # Set the sampling strategy for NearMiss
    # This will balance the majority class with the minority class
    nm = NearMiss()
    
    # Perform the undersampling
    X_resampled, y_resampled = nm.fit_resample(X, y)
    
    # Check if undersampling was successful
    if len(np.unique(y_resampled, return_counts=True)[1]) != 2 or np.unique(y_resampled, return_counts=True)[1][0] != minority_class_count:
        raise ValueError("NearMiss did not undersample correctly.")
    
    return X_resampled, y_resampled

In [323]:
X_resampled,y_resampled = oversample_with_adasyn(X_train,y_train)

In [324]:
X_train,y_train = undersample_with_nearmiss(X_resampled,y_resampled)

In [325]:
y_train.groupby(['TARGET']).size()

TARGET
0    158757
1    158757
dtype: int64

In [326]:
#X_train.to_parquet('../data/processed/X_train_engineered.parquet',index=False)
#X_test.to_parquet('../data/processed/X_test_engineered.parquet',index=False)
#y_train.to_parquet('../data/processed/y_train_engineered.parquet',index=False)
#y_test.to_parquet('../data/processed/y_test_engineered.parquet',index=False)

Scaling

<b> Pipeline Team </b> Create a function to scale and one to de-scale variables.

In [327]:
scaler = StandardScaler()

In [328]:
scaler.fit(X_train)

StandardScaler()

In [329]:
X_train_scaled = scaler.transform(X_train)
X_train = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)

In [330]:
X_test_scaled = scaler.transform(X_test)
X_test = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)

Let's try RFE

In [253]:
from sklearn.feature_selection import RFE

In [254]:
# Starting Linear Regression
logr = LogisticRegression(max_iter = 1000)
rfe = RFE(estimator=logr, n_features_to_select=70)
rfe.fit(X_train, y_train)

#Fitting Linear Regression
logr.fit(X_train,y_train)

C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

LogisticRegression(max_iter=1000)

In [256]:
X_train = X_train[rfe.get_feature_names_out()]

In [255]:
X_test = X_test[rfe.get_feature_names_out()]

### Machine Learning

M.L. Team:
- You should work on not overfitting and having a feasible model. <b> You can only use Logistic Regression/Ridge/Lasso </b>
- Think correctly on the error metric, remember that we want to have a model that predicts very well FRAUD so we can penalize False Positives
- Use my overfitted scores and try to beat them, and it is as simple as having a feasible model, after that you start to improve.

<b>LinearRegression<b>

In [257]:
# Starting Linear Regression
logr = LogisticRegression(max_iter = 10000)

#Fitting Linear Regression
logr.fit(X_train[rfe.get_feature_names_out()],y_train)

#Printing errors
print('Accuracy of training is '+ str(accuracy_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('Precision of training is '+ str(precision_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('Recall of training is '+ str(recall_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('F1 of training is '+ str(f1_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('ROC AUC of training is '+ str(roc_auc_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print()
print('Accuracy of test is '+ str(accuracy_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('Precision of test is '+ str(precision_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('Recall of test is '+ str(recall_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('F1 of test is '+ str(f1_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('ROC AUC of test is '+ str(roc_auc_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))

C:\Users\leand\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of training is 0.7540170950938214
Precision of training is 0.7550033197382149
Recall of training is 0.752083346455949
F1 of training is 0.7535405043798753
ROC AUC of training is 0.7540170950938215

Accuracy of test is 0.7330522253777614
Precision of test is 0.1008174386920981
Recall of test is 0.29741144414168935
F1 of test is 0.15058807298313387
ROC AUC of test is 0.5340603161307169


In [258]:
predicted_proba = logr.predict_proba(X_test[rfe.get_feature_names_out()])

In [259]:
predicted = logr.predict(X_test[rfe.get_feature_names_out()])

In [260]:
y_probs = predicted_proba[:, 1]

In [261]:
def find_best_threshold(y_true, y_probs):
    best_threshold = 0
    best_score = 0

    for threshold in np.arange(0.01, 1, 0.01):
        # Convert probabilities to binary predictions based on the current threshold
        y_pred = (y_probs >= threshold).astype(int)
        
        # Calculate the metric of interest, e.g., F1 score
        score = f1_score(y_true, y_pred)
        
        if score > best_score:
            best_score = score
            best_threshold = threshold

    return best_threshold, best_score

best_threshold, best_score = find_best_threshold(y_test['TARGET'], y_probs)

print(f"Best Threshold: {best_threshold}")
print(f"Best F1 Score: {best_score}")

Best Threshold: 0.38
Best F1 Score: 0.1569054122876059


Ridge

In [262]:
# Starting Ridge
logr = RidgeClassifier(alpha=0.1)

#Fitting
logr.fit(X_train[rfe.get_feature_names_out()],y_train)

#Printing errors
print('Accuracy of training is '+ str(accuracy_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('Precision of training is '+ str(precision_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('Recall of training is '+ str(recall_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('F1 of training is '+ str(f1_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print('ROC AUC of training is '+ str(roc_auc_score(y_train,logr.predict(X_train[rfe.get_feature_names_out()]))))
print()
print('Accuracy of test is '+ str(accuracy_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('Precision of test is '+ str(precision_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('Recall of test is '+ str(recall_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('F1 of test is '+ str(f1_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))
print('ROC AUC of test is '+ str(roc_auc_score(y_test,logr.predict(X_test[rfe.get_feature_names_out()]))))

C:\Users\leand\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:1209: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of training is 0.7512109549694821
Precision of training is 0.7485169491525424
Recall of training is 0.7566311201254732
F1 of training is 0.7525521631129028
ROC AUC of training is 0.751210954969482

Accuracy of test is 0.7302122401196696
Precision of test is 0.10294583465315173
Recall of test is 0.30994550408719346
F1 of test is 0.1545568803288155
ROC AUC of test is 0.5382428841772849


<b>XGBoost<b>

In [331]:
cols = ['DAYS_BIRTH','REGION_RATING_CLIENT_W_CITY','REGION_RATING_CLIENT','NAME_EDUCATION_TYPE','CODE_GENDER','DAYS_ID_PUBLISH','REG_CITY_NOT_WORK_CITY','NAME_INCOME_TYPE','FLAG_EMP_PHONE','DAYS_EMPLOYED','REG_CITY_NOT_LIVE_CITY','FLAG_DOCUMENT_3','NAME_HOUSING_TYPE','LIVE_CITY_NOT_WORK_CITY','NAME_CONTRACT_TYPE','ORGANIZATION_TYPE','FLAG_DOCUMENT_6','FLAG_WORK_PHONE','HOUR_APPR_PROCESS_START','FLAG_PHONE']

In [332]:
#X_train.columns

In [333]:
# Starting XGBoost
logr = XGBClassifier(n_estimators = 1000,learning_rate = 0.01,eval_metric='aucpr')

#Fitting
logr.fit(X_train[cols],y_train)

#Printing errors
print('Accuracy of training is '+ str(accuracy_score(y_train,logr.predict(X_train[cols]))))
print('Precision of training is '+ str(precision_score(y_train,logr.predict(X_train[cols]))))
print('Recall of training is '+ str(recall_score(y_train,logr.predict(X_train[cols]))))
print('F1 of training is '+ str(f1_score(y_train,logr.predict(X_train[cols]))))
print('ROC AUC of training is '+ str(roc_auc_score(y_train,logr.predict(X_train[cols]))))
print()
print('Accuracy of test is '+ str(accuracy_score(y_test,logr.predict(X_test[cols]))))
print('Precision of test is '+ str(precision_score(y_test,logr.predict(X_test[cols]))))
print('Recall of test is '+ str(recall_score(y_test,logr.predict(X_test[cols]))))
print('F1 of test is '+ str(f1_score(y_test,logr.predict(X_test[cols]))))
print('ROC AUC of test is '+ str(roc_auc_score(y_test,logr.predict(X_test[cols]))))

Accuracy of training is 0.8276737403705033
Precision of training is 0.86781019988263
Recall of training is 0.7731123666987912
F1 of training is 0.8177287717778741
ROC AUC of training is 0.8276737403705032

Accuracy of test is 0.8242569427883886
Precision of test is 0.09053991693585602
Recall of test is 0.13365122615803815
F1 of test is 0.10795048143053645
ROC AUC of test is 0.5088022011563679


<b>LightGBM<b>

In [334]:
# Starting XGBoost
logr = LGBMClassifier(n_estimators = 1000,learning_rate = 0.01)

#Fitting
logr.fit(X_train[cols],y_train)

#Printing errors
print('Accuracy of training is '+ str(accuracy_score(y_train,logr.predict(X_train[cols]))))
print('Precision of training is '+ str(precision_score(y_train,logr.predict(X_train[cols]))))
print('Recall of training is '+ str(recall_score(y_train,logr.predict(X_train[cols]))))
print('F1 of training is '+ str(f1_score(y_train,logr.predict(X_train[cols]))))
print('ROC AUC of training is '+ str(roc_auc_score(y_train,logr.predict(X_train[cols]))))
print()
print('Accuracy of test is '+ str(accuracy_score(y_test,logr.predict(X_test[cols]))))
print('Precision of test is '+ str(precision_score(y_test,logr.predict(X_test[cols]))))
print('Recall of test is '+ str(recall_score(y_test,logr.predict(X_test[cols]))))
print('F1 of test is '+ str(f1_score(y_test,logr.predict(X_test[cols]))))
print('ROC AUC of test is '+ str(roc_auc_score(y_test,logr.predict(X_test[cols]))))

C:\Users\leand\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leand\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 158757, number of negative: 158757
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007999 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 875
[LightGBM] [Info] Number of data points in the train set: 317514, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Accuracy of training is 0.8295697197603885
Precision of training is 0.8883923600543377
Recall of training is 0.7538439249922838
F1 of training is 0.8156063652162061
ROC AUC of training is 0.8295697197603885

Accuracy of test is 0.8442343963405381
Precision of test is 0.09213274541656997
Recall of test is 0.10817438692098093
F1 of test is 0.09951121694447926
ROC AUC of test is 0.5080170519055054


In [335]:
# Starting XGBoost
logr = RandomForestClassifier()

#Fitting
logr.fit(X_train[cols],y_train)

#Printing errors
print('Accuracy of training is '+ str(accuracy_score(y_train,logr.predict(X_train[cols]))))
print('Precision of training is '+ str(precision_score(y_train,logr.predict(X_train[cols]))))
print('Recall of training is '+ str(recall_score(y_train,logr.predict(X_train[cols]))))
print('F1 of training is '+ str(f1_score(y_train,logr.predict(X_train[cols]))))
print('ROC AUC of training is '+ str(roc_auc_score(y_train,logr.predict(X_train[cols]))))
print()
print('Accuracy of test is '+ str(accuracy_score(y_test,logr.predict(X_test[cols]))))
print('Precision of test is '+ str(precision_score(y_test,logr.predict(X_test[cols]))))
print('Recall of test is '+ str(recall_score(y_test,logr.predict(X_test[cols]))))
print('F1 of test is '+ str(f1_score(y_test,logr.predict(X_test[cols]))))
print('ROC AUC of test is '+ str(roc_auc_score(y_test,logr.predict(X_test[cols]))))

C:\Users\leand\anaconda3\Lib\site-packages\sklearn\base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy of training is 0.9999811031954496
Precision of training is 1.0
Recall of training is 0.9999622063908993
F1 of training is 0.9999811028383537
ROC AUC of training is 0.9999811031954496

Accuracy of test is 0.828235090077395
Precision of test is 0.08596183800623053
Recall of test is 0.12029972752043597
F1 of test is 0.10027254144901204
ROC AUC of test is 0.5048645162321308


Let's try switching the probability ranges

In [336]:
y_probs = predicted_proba[:, 1]

In [337]:
def find_best_threshold(y_true, y_probs):
    best_threshold = 0
    best_score = 0

    for threshold in np.arange(0.01, 1, 0.01):
        # Convert probabilities to binary predictions based on the current threshold
        y_pred = (y_probs >= threshold).astype(int)
        
        # Calculate the metric of interest, e.g., F1 score
        score = f1_score(y_true, y_pred)
        
        if score > best_score:
            best_score = score
            best_threshold = threshold

    return best_threshold, best_score

best_threshold, best_score = find_best_threshold(y_test['TARGET'], y_probs)

print(f"Best Threshold: {best_threshold}")
print(f"Best F1 Score: {best_score}")

Best Threshold: 0.38
Best F1 Score: 0.1569054122876059


In [345]:
proba = logr.predict_proba(X_test[cols])[:, 1]

threshold = 0.38

Y_pred = (proba > threshold).astype(int)

y_test


accuracy = accuracy_score(y_test, Y_pred)
precision = precision_score(y_test, Y_pred)
recall = recall_score(y_test, Y_pred)
f1 = f1_score(y_test, Y_pred)

Let's try to fine tune a model

In [346]:
accuracy

0.7508075530600299

In [347]:
precision

0.08558868703988136

In [348]:
recall

0.22016348773841962

In [349]:
f1

0.12325998245680943

Trying to set specific columns

In [156]:
cols = ['DAYS_BIRTH','REGION_RATING_CLIENT_W_CITY','REGION_RATING_CLIENT','NAME_EDUCATION_TYPE','CODE_GENDER','DAYS_ID_PUBLISH','REG_CITY_NOT_WORK_CITY','NAME_INCOME_TYPE','FLAG_EMP_PHONE','DAYS_EMPLOYED','REG_CITY_NOT_LIVE_CITY','FLAG_DOCUMENT_3','NAME_HOUSING_TYPE','LIVE_CITY_NOT_WORK_CITY','NAME_CONTRACT_TYPE','ORGANIZATION_TYPE','FLAG_DOCUMENT_6','FLAG_WORK_PHONE','HOUR_APPR_PROCESS_START','FLAG_PHONE']

In [157]:
# Starting XGBoost
logr = RandomForestClassifier()

#Fitting
logr.fit(X_train[cols],y_train)

#Printing errors
print('Accuracy of training is '+ str(accuracy_score(y_train,logr.predict(X_train[cols]))))
print('Precision of training is '+ str(precision_score(y_train,logr.predict(X_train[cols]))))
print('Recall of training is '+ str(recall_score(y_train,logr.predict(X_train[cols]))))
print('F1 of training is '+ str(f1_score(y_train,logr.predict(X_train[cols]))))
print('ROC AUC of training is '+ str(roc_auc_score(y_train,logr.predict(X_train[cols]))))
print()
print('Accuracy of test is '+ str(accuracy_score(y_test,logr.predict(X_test[cols]))))
print('Precision of test is '+ str(precision_score(y_test,logr.predict(X_test[cols]))))
print('Recall of test is '+ str(recall_score(y_test,logr.predict(X_test[cols]))))
print('F1 of test is '+ str(f1_score(y_test,logr.predict(X_test[cols]))))
print('ROC AUC of test is '+ str(roc_auc_score(y_test,logr.predict(X_test[cols]))))

KeyError: "['NAME_EDUCATION_TYPE', 'CODE_GENDER', 'REG_CITY_NOT_WORK_CITY', 'NAME_INCOME_TYPE', 'REG_CITY_NOT_LIVE_CITY', 'FLAG_DOCUMENT_3', 'NAME_HOUSING_TYPE', 'LIVE_CITY_NOT_WORK_CITY', 'NAME_CONTRACT_TYPE', 'ORGANIZATION_TYPE', 'FLAG_DOCUMENT_6', 'HOUR_APPR_PROCESS_START'] not in index"

In [158]:
X_train.columns

Index(['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'FLAG_MOBIL', 'FLAG_EMP_PHONE',
       'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL',
       'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'NAME_CONTRACT_TYPE_CASH LOANS',
       'NAME_CONTRACT_TYPE_REVOLVING LOANS', 'FLAG_OWN_CAR_N',
       'FLAG_OWN_CAR_Y', 'FLAG_OWN_REALTY_N', 'FLAG_OWN_REALTY_Y',
       'NAME_TYPE_SUITE_UNACCOMPANIED', 'NAME_INCOME_TYPE_BUSINESSMAN',
       'NAME_INCOME_TYPE_COMMERCIAL ASSOCIATE',
       'NAME_INCOME_TYPE_MATERNITY LEAVE', 'NAME_INCOME_TYPE_PENSIONER',
       'NAME_INCOME_TYPE_STATE SERVANT', 'NAME_INCOME_TYPE_STUDENT',
       'NAME_INCOME_TYPE_UNEMPLOYED', 'NAME_INCOME_TYPE_WORKING',
       'NAME_EDUCATION_TYPE_ACADEMIC DEGREE',
       'NAME_EDUCATION_TYPE_HIGHER EDUCATION',
       'NAME_EDUCATION_TYPE_INCOMPL

Let's try switching the probability ranges

In [150]:
proba = logr.predict_proba(X_test[rfe.get_feature_names_out()])[:, 1]

threshold = 0.15

Y_pred = (proba > threshold).astype(int)

y_test

# Tính các độ đo
accuracy = accuracy_score(y_test, Y_pred)
precision = precision_score(y_test, Y_pred)
recall = recall_score(y_test, Y_pred)
f1 = f1_score(y_test, Y_pred)

Let's try to fine tune a model

In [151]:
accuracy

0.49265072517180825

In [152]:
precision

0.08959880202158857

In [153]:
recall

0.5869209809264305

In [154]:
f1

0.15546453510402194

In [49]:
param_grid = {
    'num_leaves': [31, 50, 70, 100], #Number of leaves parameter        
    'max_depth': [-1, 5, 10, 20],  # Max depth parameter
    'learning_rate': [0.0001,0.001, 0.01, 0.1], #Learning Rate Parameter
    'n_estimators': [500,1000,1500,2000], #Number of fitted trees 
    'min_split_gain': [0.0, 0.1, 0.2], #Number related to minimum split gain      
    'min_child_weight': [0.001, 0.01, 0.1], #Number related to minimum child weight  
    'subsample': [0.8, 0.9, 1.0], #Subsample number           
    'colsample_bytree': [0.8, 0.9, 1.0] #number of colsample bytree
}

In [ ]:
xgb_random = XGBClassifier(random_state = 42)
# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_random, 
    param_distributions=param_grid, 
    n_iter=100,
    cv=5, 
    verbose=1, 
    random_state=42, 
    n_jobs=-1  # Use all available cores
)

# Perform the random search
random_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters found: ", random_search.best_params_)
print("Best score found: ", random_search.best_score_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
